In [51]:
import affine, concurrent, cStringIO, glob, IPython, json, os, PIL, sys, thread, time, traceback
# add binomial column to areas

# sum areas by binomial column

class SimpleProcessPoolExecutor(concurrent.futures.ProcessPoolExecutor):
    def __init__(self, max_workers):
        super(SimpleProcessPoolExecutor, self).__init__(max_workers=max_workers)
        self.futures = []
        
    def submit(self, fn, *args, **kwargs):
        future = super(SimpleProcessPoolExecutor, self).submit(fn, *args, **kwargs)
        self.futures.append(future)
        return future
    
    def get_futures(self):
        return self.futures

    def shutdown(self):
        exception_count = 0
        results = []
        for completed in concurrent.futures.as_completed(self.futures):
            try:
                results.append(completed.result())
            except Exception as e:
                exception_count += 1
                sys.stderr.write(
                    'Exception caught in SimpleProcessPoolExecutor.shutdown.  Continuing until all are finished.\n' +
                    'Exception follows:\n' +
                    traceback.format_exc())
        super(SimpleProcessPoolExecutor, self).shutdown()
        if exception_count:
            raise Exception('SimpleProcessPoolExecutor failed: %d of %d raised exception' % (exception_count, len(self.futures)))
        print 'SimpleProcessPoolExecutor succeeded: all %d jobs completed' % (len(self.futures))
        return results

class Stopwatch:
    def __init__(self, name):
        self.name = name
    def __enter__(self):
        self.start = time.time()
    def __exit__(self, type, value, traceback):
        sys.stdout.write('%s took %.1f seconds\n' % (self.name, time.time() - self.start))


In [52]:
def select_into(tname):
    c = """
    SELECT 
      m.binomial as binomial,
      sum(sum) as area    
    INTO TABLE %s_binomial_areas
    FROM %s_areas
    INNER JOIN %s as m
    ON %s_gid = m.gid
    WHERE m.presence <= 3 AND (m.code = 'CR' OR m.code = 'EN' OR m.code = 'VU' OR m.code = 'NT' OR m.code = 'LC')
    GROUP BY m.binomial
    ;
    """
    c = c % (tname, tname, tname, tname)
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd            

def drop_table(tname):
    c = """
    DROP TABLE %s_binomial_areas
    ;
    """
    c = c % (tname)
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd            

def create_index(tname):
    c = "CREATE UNIQUE INDEX IF NOT EXISTS %s_binomial_idx ON %s_binomial_areas (binomial);" % (tname, tname)
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd    

def vacuum_table(tname):
    c = "VACUUM ANALYZE %s_binomial_areas;" % tname
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd   


In [53]:
tnames = []
for f in glob.glob("../../data/redlist/*/*.shp"):
    name = os.path.basename(f).split(".")[0]
    tnames.append(name.lower())
tnames.sort()

In [54]:
exe = SimpleProcessPoolExecutor(max_workers=8)
for tname in tnames:
    exe.submit(select_into, tname)
exe.shutdown()
None

SELECT 7
SELECT 545
SELECT 84
SELECT 246
SELECT 15
SELECT 120
SELECT 444
SELECT 444
SELECT 122
SELECT 375
SELECT 633
SELECT 836
SELECT 473
SELECT 5022
SELECT 46
SELECT 638
SELECT 4746
SELECT 160
SELECT 74
SELECT 1072
SELECT 113
SELECT 123
SELECT 63
SELECT 148
SELECT 74
SELECT 127
SELECT 50
SELECT 1352
SELECT 519
SELECT 4455
SELECT 4926
SELECT 1075
SimpleProcessPoolExecutor succeeded: all 32 jobs completed


In [49]:
def test():
    c = """
    SELECT 
      mammals_gid, 
      sum
    FROM mammals_areas
    ORDER BY mammals_gid
    LIMIT 10
    """
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd            

def test1():
    c = """
    SELECT 
      mammals_gid, 
      sum,
      m.binomial,
      m.code,
      m.presence
    FROM mammals_areas
    INNER JOIN mammals as m
    ON mammals_gid = m.gid
    ORDER BY mammals_gid
    LIMIT 10
    """
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd            

def test2():
    c = """
    SELECT 
      mammals_gid, 
      sum,
      m.binomial,
      m.code,
      m.presence
    FROM mammals_areas
    INNER JOIN mammals as m
    ON mammals_gid = m.gid
    WHERE m.presence <= 3
    ORDER BY mammals_gid
    LIMIT 10
    """
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd            

def test3():
    c = """
    SELECT 
      mammals_gid, 
      sum,
      m.binomial,
      m.code,
      m.presence
    FROM mammals_areas
    INNER JOIN mammals as m
    ON mammals_gid = m.gid
    WHERE m.presence <= 3 AND (m.code = 'CR' OR m.code = 'EN' OR m.code = 'VU' OR m.code = 'NT' OR m.code = 'LC')
    ORDER BY mammals_gid
    LIMIT 20
    """
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd            

def test4():
    c = """
    SELECT 
      sum(sum),
      m.binomial
    FROM mammals_areas
    INNER JOIN mammals as m
    ON mammals_gid = m.gid
    WHERE m.presence <= 3 AND (m.code = 'CR' OR m.code = 'EN' OR m.code = 'VU' OR m.code = 'NT' OR m.code = 'LC')
    AND m.binomial = 'Ursus arctos'
    GROUP BY m.binomial
    LIMIT 20
    """
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd            


def test5():
    c = """
    SELECT 
      sum(sum),
      m.binomial
    FROM mammals_areas
    INNER JOIN mammals as m
    ON mammals_gid = m.gid
    WHERE m.presence <= 3 AND (m.code = 'CR' OR m.code = 'EN' OR m.code = 'VU' OR m.code = 'NT' OR m.code = 'LC')
    AND m.binomial = 'Panthera pardus'
    GROUP BY m.binomial
    LIMIT 20
    """
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd            


def test6():
    c = """
    SELECT 
      sum(sum),
      m.binomial
    FROM mammals_areas
    INNER JOIN mammals as m
    ON mammals_gid = m.gid
    WHERE m.presence <= 3 AND (m.code = 'CR' OR m.code = 'EN' OR m.code = 'VU' OR m.code = 'NT' OR m.code = 'LC')
    GROUP BY m.binomial
    LIMIT 20
    """
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd            
    
# binomial_areas


In [37]:
test()

 mammals_gid |       sum        
-------------+------------------
           1 | 1671888231208.87
           2 | 32436934065.5834
           3 | 15631438827921.1
           4 | 44942107589.3607
           5 | 492727889873.587
           6 | 5435242870.87612
           7 |  25551356153.122
           8 | 4487552483021.24
           9 | 110865026443.533
          10 | 42794195400.4554
(10 rows)



In [38]:
test1()

 mammals_gid |       sum        |         binomial          | code | presence 
-------------+------------------+---------------------------+------+----------
           1 | 1671888231208.87 | Chaetophractus vellerosus | LC   |        1
           2 | 32436934065.5834 | Chaetophractus vellerosus | LC   |        6
           3 | 15631438827921.1 | Ursus arctos              | LC   |        1
           4 | 44942107589.3607 | Ursus arctos              | LC   |        1
           5 | 492727889873.587 | Aotus nancymaae           | VU   |        1
           6 | 5435242870.87612 | Ursus arctos              | LC   |        1
           7 |  25551356153.122 | Ursus arctos              | LC   |        1
           8 | 4487552483021.24 | Ursus arctos              | LC   |        1
           9 | 110865026443.533 | Ursus arctos              | LC   |        1
          10 | 42794195400.4554 | Ursus arctos              | LC   |        3
(10 rows)



In [39]:
test2()

 mammals_gid |       sum        |         binomial          | code | presence 
-------------+------------------+---------------------------+------+----------
           1 | 1671888231208.87 | Chaetophractus vellerosus | LC   |        1
           3 | 15631438827921.1 | Ursus arctos              | LC   |        1
           4 | 44942107589.3607 | Ursus arctos              | LC   |        1
           5 | 492727889873.587 | Aotus nancymaae           | VU   |        1
           6 | 5435242870.87612 | Ursus arctos              | LC   |        1
           7 |  25551356153.122 | Ursus arctos              | LC   |        1
           8 | 4487552483021.24 | Ursus arctos              | LC   |        1
           9 | 110865026443.533 | Ursus arctos              | LC   |        1
          10 | 42794195400.4554 | Ursus arctos              | LC   |        3
          11 | 4407000667.81231 | Ursus arctos              | LC   |        1
(10 rows)



In [40]:
test3()

 mammals_gid |       sum        |         binomial          | code | presence 
-------------+------------------+---------------------------+------+----------
           1 | 1671888231208.87 | Chaetophractus vellerosus | LC   |        1
           3 | 15631438827921.1 | Ursus arctos              | LC   |        1
           4 | 44942107589.3607 | Ursus arctos              | LC   |        1
           5 | 492727889873.587 | Aotus nancymaae           | VU   |        1
           6 | 5435242870.87612 | Ursus arctos              | LC   |        1
           7 |  25551356153.122 | Ursus arctos              | LC   |        1
           8 | 4487552483021.24 | Ursus arctos              | LC   |        1
           9 | 110865026443.533 | Ursus arctos              | LC   |        1
          10 | 42794195400.4554 | Ursus arctos              | LC   |        3
          11 | 4407000667.81231 | Ursus arctos              | LC   |        1
          12 | 4267748809.04101 | Ursus arctos              | 

In [46]:
test4()

       sum        |   binomial   
------------------+--------------
 26277928707660.8 | Ursus arctos
(1 row)



In [48]:
test5()

       sum        |    binomial     
------------------+-----------------
 9255538235228.58 | Panthera pardus
(1 row)



In [50]:
test6()

       sum        |          binomial          
------------------+----------------------------
  6945015045255.2 | Sciurus aestuans
 5375562627.85295 | Ctenomys bergi
 49536868152.3531 | Tapirus pinchaque
 825439749676.726 | Myotis moluccarum
    7608708978123 | Paradoxurus hermaphroditus
 974851065775.895 | Plecotus kolombatovici
 4049956359060.59 | Myotis yumanensis
 15402283005.9793 | Loris tardigradus
 127971071880.446 | Otonyctomys hatti
 955768579997.909 | Neacomys paracou
 806705244554.949 | Iomys horsfieldii
 386564063966.565 | Necromys benefactus
 33302602228.0439 | Hipposideros wollastoni
  2913164400236.9 | Nyctalus lasiopterus
 35593689679.7343 | Zyzomys woodwardi
 1665160808546.24 | Dasymys rufulus
 8702708066483.73 | Myodes glareolus
 139737011770.345 | Capricornis crispus
 7373918770080.23 | Chalinolobus gouldii
 1058407807356.17 | Thaptomys nigrita
(20 rows)

